In [2]:
from pyedflib import highlevel
import os

In [7]:
def convert_signal(file):

    signals, signal_headers, header = highlevel.read_edf(file)

    print("signals: ", signals)
    print("signal_headers: ", signal_headers)

    # Find saturation and desaturation channels
    for index, item in enumerate(signal_headers):
        if item['label'] == 'SpO2':
            value_saturation = signals[index]
        if item['label'] == 'Technical':
            value_desaturation = signals[index]

    


    # Process desaturation values
    new_value_desaturation = []
    for index in range(len(value_desaturation)):
        if index % 10 == 0:
            if value_desaturation[index] != 0.0:
                new_value_desaturation.append(100)
            else:
                new_value_desaturation.append(value_desaturation[index])

    if len(new_value_desaturation) != len(value_saturation):
        raise ValueError("Length mismatch between processed desaturation and saturation signals.")

    # Calculate the area under the curve
    start_point = 0
    end_point = 0
    max_val = 0
    res = 0

    for index, value in enumerate(new_value_desaturation):
        if value != 0.0:
            if new_value_desaturation[index-1] == 0.0:
                start_point = index
                max_val = value_saturation[start_point]
            if new_value_desaturation[index+1] == 0.0:
                end_point = index
            
            if (value_saturation[index+1] > max_val) and ((index+1) != 0.0):
                max_val = value_saturation[index+1]

            if end_point > start_point:
                area_res = 0
                for i in range(start_point, end_point):
                    area_res += (max_val - value_saturation[i])
                res += area_res

    # Calculate final results
    total_duration = len(new_value_desaturation)
    processed_value = res / (120 * (total_duration / 3600))

    # Return results as a string
    result = [res, total_duration, processed_value]
    
    

    return result

In [8]:
convert_signal("./data/00002180-100709[001].edf")

signals:  [array([ 4.77607385e-03,  4.77607385e-03,  4.77607385e-03, ...,
       -4.26360113e+01, -3.72868086e+01, -3.82038148e+01],
      shape=(11242000,)), array([ 0.00477607,  0.00477607,  0.00477607, ..., -4.12175174,
        0.15761044, -3.20474556], shape=(11242000,)), array([ 4.77607385e-03,  4.77607385e-03,  4.77607385e-03, ...,
       -3.86623178e+01, -3.13262684e+01, -3.40772869e+01],
      shape=(11242000,)), array([4.77607385e-03, 4.77607385e-03, 4.77607385e-03, ...,
       7.64649424e+00, 5.81248188e+00, 3.06146334e+00], shape=(11242000,)), array([4.77607385e-03, 4.77607385e-03, 4.77607385e-03, ...,
       7.30596017e+01, 6.49593805e+01, 6.60292210e+01], shape=(11242000,)), array([ 4.77607385e-03,  4.77607385e-03,  4.77607385e-03, ...,
       -7.16745403e+01, -6.81593500e+01, -6.70895094e+01],
      shape=(11242000,)), array([ 4.77607385e-03,  4.77607385e-03,  4.77607385e-03, ...,
       -2.12392004e+01, -1.90995193e+01, -1.87938506e+01],
      shape=(11242000,)), array([

ValueError: Length mismatch between processed desaturation and saturation signals.

In [13]:
signals, signal_headers, header = highlevel.read_edf("./data/00002180-100709[001].edf")

for label, dimension in enumerate(signal_headers):
    if dimension.get('label') == 'SpO2':
        print(f"{label}: {dimension.get('label')}")
        print(dimension)
        print("signals: ", signals[label])
        
    if dimension.get('label') == 'Technical':
        print(f"{label}: {dimension.get('label')}")
        print(dimension)
        print("signals: ", signals[label])

17: SpO2
{'label': 'SpO2', 'dimension': '%', 'sample_frequency': 1.0, 'physical_max': 102.3, 'physical_min': 0.0, 'digital_max': 1023, 'digital_min': 0, 'prefilter': '', 'transducer': ''}
signals:  [95.1 95.1 95.1 ... 96.3 96.4 96.4]
43: Technical
{'label': 'Technical', 'dimension': '', 'sample_frequency': 200.0, 'physical_max': 65535.0, 'physical_min': 0.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': '', 'transducer': 'RI_Diag_Device Alice_5'}
signals:  [    0.     0.     0. ... 65535. 65535. 65535.]


In [11]:
signals, signal_headers, header = highlevel.read_edf("./data/00002180-100709[001].edf")

# print("signals: ", signals)
for label, dimension in enumerate(signal_headers):
    print(f"{label}: {dimension.get('label')}")
    print(dimension)

0: EEG C3-A2
{'label': 'EEG C3-A2', 'dimension': 'uV', 'sample_frequency': 500.0, 'physical_max': 313.0, 'physical_min': -313.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'HP:0.3Hz LP:180Hz N:50/60Hz', 'transducer': ''}
1: EEG C4-A1
{'label': 'EEG C4-A1', 'dimension': 'uV', 'sample_frequency': 500.0, 'physical_max': 313.0, 'physical_min': -313.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'HP:0.3Hz LP:180Hz N:50/60Hz', 'transducer': ''}
2: EEG A1-A2
{'label': 'EEG A1-A2', 'dimension': 'uV', 'sample_frequency': 500.0, 'physical_max': 313.0, 'physical_min': -313.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'HP:0.3Hz LP:180Hz N:50/60Hz', 'transducer': ''}
3: EEG O1-A2
{'label': 'EEG O1-A2', 'dimension': 'uV', 'sample_frequency': 500.0, 'physical_max': 313.0, 'physical_min': -313.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'HP:0.3Hz LP:180Hz N:50/60Hz', 'transducer': ''}
4: EEG O2-A1
{'label': 'EEG O2-A1', 'dimension': 'uV', '